# Multi-Person Pose Estimation

I will be creating a 

#### Step 1: Import the dependencies

In [38]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as pyplot
import numpy as np

In [39]:
# Setting memory limit if using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

### Step 2: Load the model

In [40]:
model_url = 'https://tfhub.dev/google/movenet/multipose/lightning/1'
model = hub.load(model_url)
movenet = model.signatures['serving_default']

### Step 3: drawing keypoints and edges

In [41]:
# drawing keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [42]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# drawing edges
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [43]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

### Step 3: Make detections

In [79]:
# set up webcam
webcam = cv2.VideoCapture(1)

# open webcam
while webcam.isOpened():
    ret, frame = webcam.read()

    # resize image
    image = frame.copy()
    image = tf.cast(tf.image.resize_with_pad(tf.expand_dims(image, axis=0), 288, 512), dtype=tf.int32)

    # reshape and generate keypoints
    results = movenet(image)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

    # print(keypoints_with_scores[0])

    # render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.2)

    # show image
    cv2.imshow('Movenet Multipose', frame)
    
    # Code to be executed when the 'Esc' key is pressed
    if 0x1B == cv2.waitKey(1): 
        print("Esc key pressed!")
        break


webcam.release()
cv2.destroyAllWindows()

Esc key pressed!
